In [21]:
import pandas as pd
import numpy as np
import networkx as nx
import seaborn as sns
import graphistry
import matplotlib.pyplot as plt
from web3 import Web3

In [25]:
w3 = Web3(Web3.HTTPProvider('https://mainnet.infura.io/v3/<infura-project-id>'))
w3.isConnected()

False

In [2]:
#load in data
df = pd.read_csv('transactions.csv')
df.info()

/usr/local/etc/anaconda3/envs/blockchain/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (2,13,15,22,23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481519 entries, 0 to 481518
Data columns (total 26 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   amount                     474230 non-null  float64
 1   borrowRate                 94977 non-null   float64
 2   borrowRateMode             94977 non-null   object 
 3   onBehalfOf                 474230 non-null  object 
 4   pool                       481519 non-null  object 
 5   reserve                    475230 non-null  object 
 6   timestamp                  481519 non-null  int64  
 7   user                       481519 non-null  object 
 8   type                       481519 non-null  object 
 9   reservePriceETH            474230 non-null  float64
 10  reservePriceUSD            474230 non-null  float64
 11  amountUSD                  474230 non-null  float64
 12  collateralAmount           6289 non-null    float64
 13  collateralReserve          62

In [3]:
df['onBehalfOf'].value_counts()
#0x498c5431eb517101582988fbb36431ddaac8f4b1
#0xcc9a0b7c43dc2a5f023bb9b738e45b0ef6b06e04

0x498c5431eb517101582988fbb36431ddaac8f4b1    28202
0xcc9a0b7c43dc2a5f023bb9b738e45b0ef6b06e04    15962
0xdebf20617708857ebe4f679508e7b7863a8a8eee    10977
0x63a3f444e97d14e671e7ee323c4234c8095e3516     9205
0xdcd33426ba191383f1c9b431a342498fdac73488     6754
                                              ...  
0xec71552cead53befd1eddafbfd8861bf86d677b6        1
0xda1c6e8286a5934b4f03894373b39af9b8633a6b        1
0xeaef162d982af415b552d336111722c652a20f1f        1
0x510c2801e34a149ce3f3516e0b568ccdc80aa906        1
0xb50c6b3b12fabd530ed401746feffe3f09fa2dcb        1
Name: onBehalfOf, Length: 45843, dtype: int64

In [4]:
smarts=['0xB53C1a33016B2DC2fF3653530bfF1848a515c8c5',
          '0x52D306e36E3B6B02c153d0266ff0f85d18BCD413',
          '0x7d2768dE32b0b80b7a3454c06BdAc94A69DDc7A9',
          '0xbd4765210d4167CE2A5b87280D9E8Ee316D5EC7C',
          '0x311Bb771e4F8952E6Da169b425E7e92d6Ac45756',
          '0x8A32f49FFbA88aba6EFF96F45D8BD1D4b3f35c7D',
          '0xA50ba011c48153De246E5192C8f9258A2ba79Ca9',
          '0xB9062896ec3A615a4e4444DF183F0531a77218AE',
          '0xB9062896ec3A615a4e4444DF183F0531a77218AE',
          '0x057835Ad21a177dbdd3090bB1CAE03EaCF78Fc6d',
          '0xcc9a0B7c43DC2a5F023Bb9b738E45B0Ef6B06E04',
          '0x464C71f6c2F760DdA6093dCB91C24c39e5d6e18c',
          '0xd784927Ff2f95ba542BfC824c8a8a98F3495f6b5',
           '0xAcc030EF66f9dFEAE9CbB0cd1B25654b82cFA8d5',
           '0x7937d4799803fbbe595ed57278bc4ca21f3bffcb',
           '0xbd4765210d4167ce2a5b87280d9e8ee316d5ec7c',
           '0x23a875ede3f1030138701683e42e9b16a7f87768',
           '0x8a32f49ffba88aba6eff96f45d8bd1d4b3f35c7d',
           '0xa50ba011c48153de246e5192c8f9258a2ba79ca9',
           '0xc443AD9DDE3cecfB9dfC5736578f447aFE3590ba',
           '0xcc9a0B7c43DC2a5F023Bb9b738E45B0Ef6B06E04',
           '0x464C71f6c2F760DdA6093dCB91C24c39e5d6e18c']

for i in range(len(smarts)):
    smarts[i]=smarts[i].lower()

df = df[~df['user'].isin(smarts)]
df = df[df['user']!=df['onBehalfOf']]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76684 entries, 69 to 481518
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   amount                     69395 non-null  float64
 1   borrowRate                 2669 non-null   float64
 2   borrowRateMode             2669 non-null   object 
 3   onBehalfOf                 69395 non-null  object 
 4   pool                       76684 non-null  object 
 5   reserve                    70395 non-null  object 
 6   timestamp                  76684 non-null  int64  
 7   user                       76684 non-null  object 
 8   type                       76684 non-null  object 
 9   reservePriceETH            69395 non-null  float64
 10  reservePriceUSD            69395 non-null  float64
 11  amountUSD                  69395 non-null  float64
 12  collateralAmount           6289 non-null   float64
 13  collateralReserve          6289 non-null   o

In [5]:
df['user'].value_counts()

0xdcd33426ba191383f1c9b431a342498fdac73488    15352
0xdebf20617708857ebe4f679508e7b7863a8a8eee     9802
0x63a3f444e97d14e671e7ee323c4234c8095e3516     9205
0x135896de8421be2ec868e0b811006171d9df802a     3246
0xbcc4371cc40592794bf5b727c17cf7de37ac180a     2684
                                              ...  
0xf75a0bf2cb8c87ce64c4d379e4c9e55e7dec87be        1
0x02ac45ae315fd6fdab7099e520b3d0849da83a8b        1
0x585de4a1f910540345abd828e0e81d0d03a113ed        1
0xe11ad4d81aac2fcdbf123f7a08286c265fd1f827        1
0x0999033a70b936bd10582437040550eab875ca95        1
Name: user, Length: 8659, dtype: int64

In [6]:
df['onBehalfOf'].value_counts()

0x498c5431eb517101582988fbb36431ddaac8f4b1    14101
0xe069cb01d06ba617bcdf789bf2ff0d5e5ca20c71     3984
0xcc9a0b7c43dc2a5f023bb9b738e45b0ef6b06e04     2838
0xfd3dfb524b2da40c8a6d703c62be36b5d8540626     1559
0xdcd33426ba191383f1c9b431a342498fdac73488     1513
                                              ...  
0xa5367338a7ed2e1387f762aa0fac9275b917690d        1
0x4bb34bb979239f69d01c2a558cebb58eee13ffa1        1
0x43b30be5cec9fd73ebf4922b21ee9e63ac178311        1
0x2db3d4df23069a937ddd42f5fb5cab24032ca007        1
0x9bc40dc6d5753de0a0eaf02c2b891ae5c95ebbb2        1
Name: onBehalfOf, Length: 13156, dtype: int64

In [7]:
all_users = set(df['user']).union(set(df['onBehalfOf']))
i=0
user_map = dict()
for user in all_users:
    user_map[user]=i
    i+=1

In [8]:
df['user'] = df['user'].map(user_map)
df['onBehalfOf'] = df['onBehalfOf'].map(user_map)

In [9]:
graphistry.register(api=3,username='Chris95cam',password='x@9DePmTfLPi@2a')

In [10]:
g = graphistry.edges(df,'onBehalfOf','user')

In [11]:
g.plot()

In [12]:
df.head()

,amount,borrowRate,borrowRateMode,onBehalfOf,pool,reserve,timestamp,user,type,reservePriceETH,...,reservePriceETHPrincipal,reservePriceUSDPrincipal,reservePriceETHCollateral,reservePriceUSDCollateral,amountUSDPincipal,amountUSDCollateral,borrowRateModeFrom,borrowRateModeTo,stableBorrowRate,variableBorrowRate
69,14.00,3.068973,Variable,16107,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,WETH,1612652597,9658,borrow,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
220,12.27,0.417700,Variable,7610,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,WETH,1609511236,9658,borrow,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
222,8.00,0.160707,Variable,16391,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,WETH,1607142253,9658,borrow,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
247,30.00,1.609256,Variable,11989,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,WETH,1613532571,9658,borrow,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
338,4.00,2.617187,Variable,8613,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,WETH,1615704565,9658,borrow,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df['user'].value_counts()

9658     15352
16857     9802
1141      9205
16819     3246
1411      2684
         ...  
5534         1
3485         1
9624         1
15707        1
6179         1
Name: user, Length: 8659, dtype: int64